In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


In [ ]:
! pip install transformers

In [ ]:
! pip install -U transformers

In [ ]:
! pip install transformers[torch]

In [ ]:
!pip install memory_profiler

In [ ]:
import torch, gc, random
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
%load_ext memory_profiler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from torch import nn
from torch.utils.data import DataLoader


In [ ]:
!pip install einops

In [ ]:
!pip install pytorch_lamb

In [ ]:
from pytorch_lamb import Lamb

In [ ]:
def set_seed(seed: int):
  random.seed(seed)
  np.random.seed(seed)
  if is_torch_available():
      torch.manual_seed(seed)
      torch.cuda.manual_seed_all(seed)

  if is_tf_available():
      import tensorflow as tf

      tf.random.set_seed(seed)

set_seed(42)

In [ ]:
device = torch.device ('cuda' if torch.cuda.is_available () else 'cpu')
device

In [ ]:
os.listdir('/kaggle/input/')

# Loading Data

In [ ]:
train=pd.read_csv('/kaggle/input/stanford-ribonanza-rna-folding/train_data.csv')

In [ ]:
train2=train[train['SN_filter']==1]

In [ ]:
df_2A3_MaP=train2[train2['experiment_type'] == '2A3_MaP']
reactivity_columns = df_2A3_MaP.columns[df_2A3_MaP.columns.str.startswith('reactivity_0')]
df_2A3_MaP['react_2A3']=df_2A3_MaP.apply(lambda row: row.loc[reactivity_columns].values.astype('float32'), axis=1)
df_2A3_MaP = df_2A3_MaP.drop(columns=reactivity_columns).reset_index(drop=True)

df_DMS_MaP=train2[train2['experiment_type'] == 'DMS_MaP']
reactivity_columns2 = df_DMS_MaP.columns[df_DMS_MaP.columns.str.startswith('reactivity_0')]
df_DMS_MaP['react_DMS']=df_DMS_MaP.apply(lambda row: row.loc[reactivity_columns2].values.astype('float32'), axis=1)
df_DMS_MaP = df_DMS_MaP.drop(columns=reactivity_columns2).reset_index(drop=True)



In [ ]:
def filter_col(data):
  error_columns = [col for col in data.columns if 'error' in col]
  data = data.drop(columns=error_columns).reset_index(drop=True)
  data = data.drop(columns=['reads', 	'signal_to_noise', 	'SN_filter','dataset_name']).reset_index(drop=True)
  return data

In [ ]:
def filter_col2(data, cols_to_filter):
  data = data.drop(columns=cols_to_filter).reset_index(drop=True)
  return data

In [ ]:
df_DMS_MaP=filter_col(df_DMS_MaP)
df_2A3_MaP=filter_col(df_2A3_MaP)

In [ ]:
merge_df=df_2A3_MaP.merge(df_DMS_MaP, on='sequence_id', how='inner')

In [ ]:
merge_df.columns

In [ ]:
df1= filter_col2(merge_df,['experiment_type_x', 'sequence_y', 'experiment_type_y'])

In [ ]:
# Specify the number of rows you want in your random chunk
chunk_size = 5000  # Change this to the desired chunk size
# Randomly select a chunk of the DataFrame
df = df1.sample(n=chunk_size)
# Specify the file pathway where you want to save the CSV file
#file_pathway = '/content/drive/MyDrive/Kaggle_competition/chunk3.csv'
df=df.reset_index(drop=True)

# Save the DataFrame to a CSV file
#df.to_csv(file_pathway, index=False)  # Set index=False to exclude the index column from the CSV

In [ ]:
# need to replace U with T as we are going to use DNABERT model
def replace_U_with_T(s):
    # s is a string that contains U
    # return a new string that replaces U with T
    return s.replace('U', 'T')


In [ ]:
df['sequence']=df['sequence_x'].apply(lambda x: replace_U_with_T(x))

In [ ]:
def padding_nan(arr, pad_len):
  pad_width = (0, pad_len - len(arr)) # The number of values to add before and after the array
  pad_value = np.nan # The value to use for padding
  padded_arr = np.pad(arr, pad_width, mode="constant", constant_values=pad_value)
  return(padded_arr)

In [ ]:
# need to do padding to 457 as it is a max length of the RNA in test file see below
df['react_2A3_pad']=df['react_2A3'].apply(lambda x: padding_nan(x,457))

In [ ]:
df['react_DMS_pad']=df['react_DMS'].apply(lambda x: padding_nan(x,457))

In [ ]:
X1 = df.sequence.astype(str)
y1 = df.react_2A3_pad
y2 = df.react_DMS_pad


# Split Data
X_train2A3, X_test2A3, y_train2A3, y_test2A3 = train_test_split(X1.tolist(), y1, test_size=0.15)
X_trainDMS, X_testDMS, y_trainDMS, y_testDMS = train_test_split(X1.tolist(), y2, test_size=0.15)


x_train2A3 = np.array(X_train2A3)
x_test2A3 = np.array(X_test2A3)

x_trainDMS = np.array(X_trainDMS)
x_testDMS = np.array(X_testDMS)



print(x_trainDMS.shape)
print(y_trainDMS.shape)
print(x_trainDMS[0][:50])

print(x_train2A3.shape)
print(y_train2A3.shape)
print(x_train2A3[0][:50])

In [ ]:
df_2A3_train = pd.DataFrame({"sequence": x_train2A3, "react": y_train2A3}).reset_index(drop=True)
df_2A3_test = pd.DataFrame({"sequence": x_test2A3, "react": y_test2A3}).reset_index(drop=True)

df_DMS_train = pd.DataFrame({"sequence": x_trainDMS, "react": y_trainDMS}).reset_index(drop=True)
df_DMS_test = pd.DataFrame({"sequence": x_testDMS, "react": y_testDMS}).reset_index(drop=True)

## Test set

In [ ]:
test_df=pd.read_csv('/kaggle/input/stanford-ribonanza-rna-folding/test_sequences.csv')

In [ ]:
test_df['len']=test_df['sequence'].apply(lambda x: len(x))

In [ ]:
max(test_df['len']) # as I understand the max len output should be 457

# Loading DNABERT

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForPreTraining, BertConfig, BertForPreTraining

tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M")

In [ ]:
#testing model
rna=df['sequence'][0]
inputs = tokenizer(rna, return_tensors = 'pt',padding=True, truncation=True, max_length=38)["input_ids"]
inputs

In [ ]:
#according to vocabulary https://huggingface.co/zhihan1996/DNABERT-2-117M/blob/main/tokenizer.json token for "[CLS]": 1 which has index 0
hidden_state = model(inputs)[0] # pretrained model
hidden_state[:, 0, :] # or replace by mean pooling

# Dataset

In [ ]:
class RNA_Dataset:
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        inputs = tokenizer(self.data.sequence.iloc[idx], return_tensors = 'pt', padding=True, truncation=True, max_length=38)["input_ids"]
        hidden_state = model(inputs)[0] # pretrained model
        X = hidden_state[:, 0, :] # or replace by mean pooling
        y = self.data.react.iloc[idx]
        return X, y

In [ ]:
Dataset_2A3train=RNA_Dataset(df_2A3_train)
Dataset_2A3test=RNA_Dataset(df_2A3_test)

In [ ]:
Dataset_DMStrain=RNA_Dataset(df_DMS_train)
Dataset_DMStest=RNA_Dataset(df_DMS_test)

In [ ]:
batch_size = 10

# Create data loaders.
train_dataloader_DMS = DataLoader(Dataset_DMStrain, batch_size=batch_size)
test_dataloader_DMS = DataLoader(Dataset_DMStest, batch_size=batch_size)

for X, y in train_dataloader_DMS:
    print(X.shape)
    print(y.shape, y.dtype)
    break

In [ ]:
batch_size = 10

# Create data loaders.
train_dataloader = DataLoader(Dataset_2A3train, batch_size=batch_size)
test_dataloader = DataLoader(Dataset_2A3test, batch_size=batch_size)

for X, y in train_dataloader:
    print(X.shape)
    print(y.shape, y.dtype)
    break

# Model

In [ ]:
class BertRegressor(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(nn.Linear(768, 512), nn.GELU(),
                                    nn.Linear(512, 128), nn.GELU(),
                                    nn.Linear(128, 128), nn.GELU(),
                                    nn.Linear(128, 457))

    def forward(self, cls_embedding):
        y = self.layers(cls_embedding)
        return y

In [ ]:
class CustomMSELoss(nn.Module):
  def __init__(self):
    super(CustomMSELoss, self).__init__()

  def forward(self, y_pred, y_true):
    # Check if the target values are nan
    nan_mask = torch.isnan(y_true)
    # Replace the nan values with zeros
    y_true = torch.where(nan_mask, torch.zeros_like(y_true), y_true)
    # Calculate the absolute differences between the predicted and true values
    diff = torch.square(y_pred - y_true)
    # Mask out the nan values from the differences
    diff = torch.where(nan_mask, torch.zeros_like(diff), diff)
    # Calculate the mean absolute error while ignoring the nan values
    mae = torch.mean(diff)
    return mae


In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches

    print(f" Avg loss: {test_loss:>8f} \n")

In [ ]:
reg_head_2A3= BertRegressor().to(device)

In [ ]:
loss_fn=CustomMSELoss()
#optimizer = torch.optim.SGD(reg_head_2A3.parameters(), lr=0.001, momentum=0.9)
optimizer = Lamb(reg_head_2A3.parameters(), lr=0.001)
optimizer_bert = Lamb(model.parameters(), lr=0.001)

In [ ]:
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader,reg_head_2A3, loss_fn, optimizer)
    test(test_dataloader, reg_head_2A3, loss_fn)
    #test(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
epochs = 2
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader,reg_head_2A3, loss_fn, optimizer_bert)
    test(test_dataloader, reg_head_2A3, loss_fn)
    #test(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
epochs = 4
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader,reg_head_2A3, loss_fn, optimizer)
    test(test_dataloader, reg_head_2A3, loss_fn)
    #test(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
reg_head_DMS= BertRegressor().to(device)
#optimizer2=torch.optim.SGD(reg_head_DMS.parameters(), lr=0.001, momentum=0.9)
optimizer2 = Lamb(reg_head_DMS.parameters(), lr=0.001)

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader_DMS,reg_head_DMS, loss_fn, optimizer2)
    test(test_dataloader_DMS, reg_head_DMS, loss_fn)
    #test(test_dataloader, model, loss_fn)
print("Done!")

# Submission file

In [ ]:
#test file was loaded before
test_df.head()
test_df['sequence']=test_df['sequence'].dropna().astype('str')
test_df['sequence']=test_df['sequence'].apply(lambda x: replace_U_with_T(x))

In [ ]:
class test_Dataset:
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        inputs = tokenizer(self.data.sequence.iloc[idx], return_tensors = 'pt', padding=True, truncation=True, max_length=38)["input_ids"]
        hidden_state = model(inputs)[0] # pretrained model
        X = hidden_state[:, 0, :] # or replace by mean pooling
        return X


In [ ]:
test_Data=test_Dataset(test_df)
test_loader = DataLoader(test_Data, batch_size=32,num_workers=8, pin_memory=True)

In [ ]:
# I calculated this in chunks, model is heavy pruning of model needed or acsess to several GPU
reg_head_2A3.eval()
reg_head_DMS.eval()
predict_2A3 = []
predict_DMS = []
with torch.no_grad():
  for batch,X in enumerate(test_loader):
    X=X.to(device)
    predict_2A3.append(reg_head_2A3(X))
    predict_DMS.append(reg_head_DMS(X))

In [ ]:
react=predict_DMS[0].squeeze().cpu().numpy()
for i in range(1,len(predict_DMS)):
  react_i=predict_DMS[i].squeeze().cpu().numpy()
  react=np.concatenate((react_i, react), axis=0)

In [ ]:
pred_DMS=[]
for i in range(react.shape[0]):
  pred_DMS.append(react[i][:test_df['len'][i]])
pred_DMS_by_id=np.concatenate(pred_DMS)

In [ ]:
react2=predict_2A3[0].squeeze().cpu().numpy()
for i in range(1,len(predict_2A3)):
  react_i=predict_2A3[i].squeeze().cpu().numpy()
  react2=np.concatenate((react_i, react2), axis=0)

In [ ]:
pred_2A3=[]
for i in range(react2.shape[0]):
  pred_2A3.append(react2[i][:test_df['len'][i]])
pred_2A3_by_id=np.concatenate(pred_2A3)

In [ ]:
sub=pd.DataFrame({
        "id": np.arange(0, len(pred_DMS_by_id), 1),
        "reactivity_DMS_MaP": pred_DMS_by_id, "reactivity_2A3_MaP": pred_2A3_by_id
    })

In [ ]:
sub.to_csv('/kaggle/working/submission.csv', index=False)